In [2]:
%load_ext lab_black

In [3]:
import kfp
from kfp.components import InputPath, OutputPath
import kfp.dsl as dsl

In [7]:
BASE_IMAGE = (
    "quay.io/ibm/kubeflow-notebook-image-ppc64le:elyra3.13.0-py3.8-tf2.9.2-pt1.12.1-v0"
)


def configure_tensorboard(
    mlpipeline_ui_metadata_path: OutputPath(str),
    pvc_name: str,
    pvc_path: str = "",
    tensorboard_name: str = "",
) -> None:
    """
    Monitors a training job based on Tensorboard logs.
    Logs are expected to be written to the specified subpath of the pvc

    Params:
    mlpipeline_ui_metadata_path - Kubeflow provided path for visualizations
                                  The visualization contains a link to the deployed tensorboard service
    pvc_name: str - the name of the pvc where the logs are stored
    pvc_path: str - the path to the logs on the pvc. This path should NOT include any mount point.
                    So for example if the traning component mounts the pvc as "/workspace" and the logs are written to
                    "/workspace/tensorboard_logs", you should only provide "tensorborad_logs" for this param.
    tensorboard_name: str - the name of the tensorboard, if empty, the name of the workflow is used.
    """
    from collections import namedtuple
    import json
    from kubernetes import client, config, watch
    import logging
    import sys
    import os
    import yaml
    import textwrap
    import json
    import http

    logging.basicConfig(
        stream=sys.stdout,
        level=logging.INFO,
        format="%(levelname)s %(asctime)s: %(message)s",
    )
    logger = logging.getLogger()

    if not tensorboard_name:
        tensorboard_name = "{{workflow.name}}"

    namespace = "{{workflow.namespace}}"

    config.load_incluster_config()
    api_client = client.ApiClient()
    apps_api = client.AppsV1Api(api_client)
    custom_object_api = client.CustomObjectsApi(api_client)

    # Delete possible existing tensorboard
    try:
        custom_object_api.delete_namespaced_custom_object(
            group="tensorboard.kubeflow.org",
            version="v1alpha1",
            plural="tensorboards",
            namespace=namespace,
            name=tensorboard_name,
            body=client.V1DeleteOptions(),
        )
    except client.exceptions.ApiException as e:
        if e.status != http.HTTPStatus.NOT_FOUND:
            raise

    tensorboard_spec = textwrap.dedent(
        f"""\
            apiVersion: tensorboard.kubeflow.org/v1alpha1
            kind: Tensorboard
            metadata:
              name: "{tensorboard_name}"
              namespace: "{namespace}"
              ownerReferences:
                - apiVersion: v1
                  kind: Workflow
                  name: "{{workflow.name}}"
                  uid: "{{workflow.uid}}"
            spec:
              logspath: "pvc://{pvc_name}/{pvc_path}"
            """
    )

    logger.info(tensorboard_spec)

    custom_object_api.create_namespaced_custom_object(
        group="tensorboard.kubeflow.org",
        version="v1alpha1",
        plural="tensorboards",
        namespace=namespace,
        body=yaml.safe_load(tensorboard_spec),
        pretty=True,
    )

    tensorboard_watch = watch.Watch()
    try:
        for tensorboard_event in tensorboard_watch.stream(
            custom_object_api.list_namespaced_custom_object,
            group="tensorboard.kubeflow.org",
            version="v1alpha1",
            plural="tensorboards",
            namespace=namespace,
            field_selector=f"metadata.name={tensorboard_name}",
            timeout_seconds=0,
        ):

            logger.info(f"tensorboard_event: {json.dumps(tensorboard_event, indent=2)}")

            if tensorboard_event["type"] == "DELETED":
                raise RuntimeError("The tensorboard was deleted!")

            tensorboard = tensorboard_event["object"]

            if "status" not in tensorboard:
                continue

            deployment_state = "Progressing"
            if "conditions" in tensorboard["status"]:
                deployment_state = tensorboard["status"]["conditions"][-1][
                    "deploymentState"
                ]

            if deployment_state == "Progressing":
                logger.info("Tensorboard deployment is progressing...")
            elif deployment_state == "Available":
                logger.info("Tensorboard deployment is Available.")
                break
            elif deployment_state == "ReplicaFailure":
                raise RuntimeError(
                    "Tensorboard deployment failed with a ReplicaFailure!"
                )
            else:
                raise RuntimeError(f"Unknown deployment state: {deployment_state}")
    finally:
        tensorboard_watch.stop()

    button_style = (
        "align-items: center; "
        "appearance: none; "
        "background-color: rgb(26, 115, 232); "
        "border: 0px none rgb(255, 255, 255); "
        "border-radius: 3px; "
        "box-sizing: border-box; "
        "color: rgb(255, 255, 255); "
        "cursor: pointer; "
        "display: inline-flex; "
        "font-family: 'Google Sans', 'Helvetica Neue', sans-serif; "
        "font-size: 14px; "
        "font-stretch: 100%; "
        "font-style: normal; font-weight: 700; "
        "justify-content: center; "
        "letter-spacing: normal; "
        "line-height: 24.5px; "
        "margin: 0px 10px 2px 0px; "
        "min-height: 25px; "
        "min-width: 64px; "
        "padding: 2px 6px 2px 6px; "
        "position: relative; "
        "tab-size: 4; "
        "text-align: center; "
        "text-indent: 0px; "
        "text-rendering: auto; "
        "text-shadow: none; "
        "text-size-adjust: 100%; "
        "text-transform: none; "
        "user-select: none; "
        "vertical-align: middle; "
        "word-spacing: 0px; "
        "writing-mode: horizontal-tb;"
    )

    # See: https://github.com/kubeflow/kubeflow/blob/master/components/crud-web-apps/tensorboards/frontend/src/app/pages/index/index.component.ts
    # window.open(`/tensorboard/${tensorboard.namespace}/${tensorboard.name}/`);
    ui_address = f"/tensorboard/{namespace}/{tensorboard_name}/#scalars"

    markdown = textwrap.dedent(
        f"""\
        # Tensorboard
        - <a href="{ui_address}" style="{button_style}" target="_blank">Connect</a>
        - <a href="/_/tensorboards/" style="{button_style}" target="_blank">Manage all</a>
        """
    )

    markdown_output = {
        "type": "markdown",
        "storage": "inline",
        "source": markdown,
    }

    ui_metadata = {"outputs": [markdown_output]}
    with open(mlpipeline_ui_metadata_path, "w") as metadata_file:
        json.dump(ui_metadata, metadata_file)

    logging.info("Finished.")


configure_tensorboard_comp = kfp.components.create_component_from_func(
    func=configure_tensorboard,
    base_image=BASE_IMAGE,
    packages_to_install=["kubernetes"],
    output_component_file="configure_tensorboard_component.yaml",
)